<a href="https://colab.research.google.com/github/Whereamiactually/lyceumcompling11/blob/main/Spellchecking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исправление опечаток

(Спасибо Михаилу Нефёдову, автору материалов, и Татьяне Казаковой)

In [1]:
!pip install razdel tqdm

In [2]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

In [3]:
# библиотека для отслеживания прогресса
from tqdm.notebook import tqdm

Возьмем данные с соревнования [Dialog Evaluation 2016](http://www.dialog-21.ru/evaluation/2016/spelling_correction/) по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

(Сейчас из данных удалены случаи, когда в словах пропущен или вставлен пробел, чтобы было проще сопоставить слова в предложении.)

In [4]:
! wget https://raw.githubusercontent.com/tbkazakova/compling_for_lyceum/main/data/sents_with_mistakes.txt

--2023-10-06 10:17:30--  https://raw.githubusercontent.com/tbkazakova/compling_for_lyceum/main/data/sents_with_mistakes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122030 (119K) [text/plain]
Saving to: ‘sents_with_mistakes.txt’

sents_with_mistakes 100%[===================>] 119.17K  --.-KB/s    in 0.02s   

2023-10-06 10:17:30 (6.20 MB/s) - ‘sents_with_mistakes.txt’ saved [122030/122030]



In [5]:
! wget https://raw.githubusercontent.com/tbkazakova/compling_for_lyceum/main/data/correct_sents.txt

--2023-10-06 10:19:30--  https://raw.githubusercontent.com/tbkazakova/compling_for_lyceum/main/data/correct_sents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119546 (117K) [text/plain]
Saving to: ‘correct_sents.txt’

correct_sents.txt   100%[===================>] 116.74K  --.-KB/s    in 0.02s   

2023-10-06 10:19:30 (6.89 MB/s) - ‘correct_sents.txt’ saved [119546/119546]



In [6]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
good = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [7]:
len(good)

915

In [9]:
# Посмотрим на пары предложений
print(bad[4])
print(good[4])

в массе своей они конечно все оччччень милые )
в массе своей они конечно все очень милые


In [ ]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

In [ ]:
pprint(align_words(good[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [ ]:
mistakes = []
total = 0
for i in range(len(good)):
    word_pairs = align_words(good[i], bad[i])


    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [ ]:
mistakes[:5]

[('симпатичнейшее', 'симпатичнейшое'),
 ('апофеозом', 'опофеозом'),
 ('поясним', 'пояним'),
 ('получатся', 'полчатся'),
 ('очень', 'оччччень')]

In [ ]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.12886443221610805


Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [ ]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Тексты из википедии для этого хорошо подходят.

In [ ]:
corpus = open('wiki_data.txt', encoding='utf8').read()

Попробуем предсказать ошибку простым заглядыванием в словарь. Если слово не в словаре - оно неправильное.

In [ ]:
# создаем словарь
vocab = Counter(re.findall('\w+', corpus.lower()))

In [ ]:
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [ ]:
# для оценки создаем два списка y_true и y_pred
# проходимся по предложениям
# сопоставляем слова с помощью функции align_words
# проходимся по парам слов и
# если слова одинаковые добавляем в y_true 0
# если слова разные добавляем в y_true 1
# предказываем ошибочность слова из bad списка
# добавляем предсказание в список y_pred

y_true = []
y_pred = []

for i in range(len(good)):
    word_pairs = align_words(good[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)

        y_pred.append(predict_mistaken(pair[1], vocab))


In [ ]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred, ))

              precision    recall  f1-score   support

           0       0.98      0.91      0.94      8707
           1       0.59      0.88      0.71      1288

    accuracy                           0.91      9995
   macro avg       0.79      0.90      0.83      9995
weighted avg       0.93      0.91      0.91      9995



### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известного алгоритма Питера Норвига. Идея алгоритма очень простая - для каждого неправильного слова нужно сгенерировать варианты исправлений и выбрать из них тот, что есть в словаре, а если таких несколько, то выбрать наиболее вероятный.

Неправильными считаются слова, которых нет в словаре (также как в функции выше). А вероятность слова расчитывается по формуле - абсолютная частота слова в корпусе разделить на количество слов в корпусе.

Абсолютные частоты лежат в счетчике

In [ ]:
vocab.most_common(10)

[('в', 267296),
 ('и', 147115),
 ('на', 81926),
 ('с', 61681),
 ('года', 43894),
 ('по', 37235),
 ('году', 32197),
 ('из', 29150),
 ('был', 23293),
 ('не', 23228)]

В вероятности они преобразуются вот такой функцией

In [ ]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N


In [ ]:
print(P('солнце')) # вероятность слова солнце по нашему словарю

2.4440966240624417e-05


(для того, чтобы вывести привычную десятичную дробь, можно воспользоваться строковым методом .format с указанием количества знаков после запятой)

In [ ]:
print('{0:.15f}'.format(P('солнце')))

0.000024440966241


Теперь самое интересное - способ генерации вариантов исправлений. Они генерируются с помощью: удаления, перестановки, замены, вставки.

1) удаление - по очереди выбрасываем из слова 1 букву (слово - лово, сово, слво, слоо, слов)  
2) перестановка - по очереди меняем соседние буквы (слово - лсово, солво, слвоо, слоов)  
3) замена - по очереди заменям каждую букву на другую букву алфавита (слово - алово, блово, влово, глово...)  
4) вставка - по очереди вставляем между соседними буквами букву алфавита (слово - салово, сблово, свлово, сглово...)  

В алгоритма два уровня генерации - сначала генерируются варианты для оригинального слова, а потом варианты для каждого варианта. Таким образом, максимальное допустимое отличие для ошибки и предсказания - 2 буквы.

In [ ]:
# оригинальный код вот тут - https://norvig.com/spell-correct.html

def correction(word):
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word):
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in vocab)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

Попробуем исправить

In [ ]:
%%time
correction('ссонлце')

CPU times: user 183 ms, sys: 948 µs, total: 184 ms
Wall time: 187 ms


'солнце'

In [ ]:
%%time
correction('ваще')

CPU times: user 1.78 ms, sys: 0 ns, total: 1.78 ms
Wall time: 7.23 ms


'чаще'

In [ ]:
%%time
correction('опофеоз')

CPU times: user 649 µs, sys: 0 ns, total: 649 µs
Wall time: 661 µs


'апофеоз'

Выводов по единичным примерам не сделаешь, поэтому давайте запустим на всем нашем корпусе

Для оценки используем будем использовать три метрики:  
1) процент правильных слов;  
2) процент исправленных ошибок  
3) процент ошибочно исправленных правильных слов

In [ ]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(good))):
    word_pairs = align_words(good[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше

        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted


        if predicted == pair[0]:
            correct += 1
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1


  0%|          | 0/915 [00:00<?, ?it/s]

Получается, что в целом не стало лучше. Хотя 50% опечаток исправляются корректно

In [ ]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8708354177088544
0.5116459627329193
0.07603077983231882


Ещё проблема тут в том, что алгоритм медленно работает для длинных слов.

In [ ]:
%%time
correction('солнвце')

CPU times: user 722 µs, sys: 0 ns, total: 722 µs
Wall time: 892 µs


'солнце'

In [ ]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 3.34 s, sys: 5.82 ms, total: 3.35 s
Wall time: 3.39 s


'насмехатьсяаававттававаываываы'

Посмотрим, как исправляются самые частотные ошибки.

In [ ]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'седая'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'чаще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кстати'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'факто'),
 ('очень', 'оооочень', 'сорочень'),
 ('это', 'ето', 'что')]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования

Самое известное расстояние редактирования - расстояние Левенштейна. Тут мы не будет поднобно разбирать алгоритм, можете почитать [тут](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0), посмотреть более понятный разбор [тут](https://www.youtube.com/watch?v=MiqoA-yF-0M), а код на питоне есть [тут](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%BE%D0%B2/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0).
Про самого Левенштейна можно почитать вот тут - https://nplus1.ru/material/2017/09/25/vladimir-levenshtein

Основная идея - найти минимальное число исправлений, которое нужно сделать в слове А, чтобы получить слово Б. Причем допустимы только три вида исправлений - удаление, вставка, замена.

Ещё есть расстояние Дамерау-Левенштейна - почти то же самое, только разрешена ещё операция перестановки.

Есть библиотека textdistance, в которой реализованы многие методы нахождения расстояний.

In [ ]:
!pip install textdistance

In [ ]:
import textdistance

In [ ]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()

    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)

    return similarities.most_common(topn)

In [ ]:
%%time
get_closest_match_with_metric('опофеоз', vocab, 5, textdistance.damerau_levenshtein)

CPU times: user 56 s, sys: 133 ms, total: 56.1 s
Wall time: 1min 10s


[('апофеоз', 0.8571428571428572),
 ('апофеоза', 0.75),
 ('апофеозом', 0.6666666666666667),
 ('апофеты', 0.5714285714285714),
 ('опорной', 0.5714285714285714)]

In [ ]:
%%time
get_closest_match_with_metric('сонце', vocab, 3, textdistance.damerau_levenshtein)

CPU times: user 35.2 s, sys: 79 ms, total: 35.3 s
Wall time: 37.4 s


[('солнце', 0.8333333333333334), ('конце', 0.8), ('монце', 0.8)]

Спойлер: не сильно лучше работает.

## Готовые инструменты

Есть несколько готовых опечаточников:  
1) Hunspell - https://pypi.org/project/hunspell/  
2) Jamspell - https://github.com/bakwc/JamSpell#python  
3) Яндекс.Спеллер - https://yandex.ru/dev/speller/

Если вам понадобится в серьезной задаче исправлять опечатки, то начните с них, а не с алгоритма Норвига.

Для Яндекс Спеллера есть питоновская библиотека, которая упрощает его использование. У него есть некоторые ограничения (10 к запросов в день), но для небольших проектов этого вполне достаточно.

In [ ]:
!pip install pyaspeller

In [ ]:
from pyaspeller import YandexSpeller
speller = YandexSpeller()

In [ ]:
# Так можно исправить предложение целиком
fixed = speller.spelled('Нипрвильное приджлажение.')
fixed

'Неправильное предложение.'

In [ ]:
# А так проверить и исправить отдельное слово
from pyaspeller import Word
check = Word('приджлажение')
print(check.text, check.correct, check.spellsafe, check.variants)

/usr/local/lib/python3.10/dist-packages/pyaspeller/word.py:17: UserWarning: Class Word is deprecated. Use YandexSpeller().spelled(text) instead
  warnings.warn("Class Word is deprecated. Use YandexSpeller().spelled(text) instead")


приджлажение False предложение ['предложение', 'придлажение', 'приджложение']


In [ ]:
fixed = speller.spelled("I'm not sleapy and tehre is no place I'm giong to.")
fixed

"I'm not sleepy and there is no place I'm going to."

И ещё один посмотрим

In [ ]:
! pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=1716a03fed2e1e3ea9a457c2cc2b080e887a06e1f06023e32667ee8d0e030387
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [ ]:
from autocorrect import Speller

In [ ]:
spell = Speller(lang='en')

In [ ]:
spell("I'm not sleapy and tehre is no place I'm giong to.")

"I'm not sleepy and there is no place I'm going to."

In [ ]:
%timeit spell("I'm not sleapy and tehre is no place I'm giong to.")

521 µs ± 174 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
# Можно ускорить
spell = Speller(fast=True)

In [ ]:
%timeit spell("I'm not sleapy and tehre is no place I'm giong to.")

390 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


А для русского?

In [ ]:
spell = Speller(lang='ru')

In [ ]:
spell('Нипрвильное приджлажение.')

'Првильное приджлажение.'

Ну вот, было неправильное, стало правильное.

(в общем, используйте YandexSpeller для русского)